In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# **데이터를 다시 사용하고싶으면 다시 등록해야함 
# df.drop을 객체를 지정하지 않고 사용해서 데이터가 날라감 
- 다른 변수를 이용해서 학습하고 싶으면 넣고싶으면 아래의 코드를 다시 실행해야함

In [8]:
df=pd.read_csv('부상 데이터셋.csv', encoding='UTF-8')

In [9]:
df
df.columns

print(df['Injury Location'].unique())
print(df['Incident Type'].unique())
#df.count()

['Trunk' 'Abdomen' 'Back' 'Legs' 'Neck' 'Head' 'Multiple' 'Eye' nan 'Arms'
 'Hands' 'Feet']
['Burn' 'Cut' 'Lifting' 'Fall' 'Falling object' 'Crush & Pinch'
 'Slip/trip' 'Vehicle' 'Equipment']


In [10]:
df.dropna(inplace = True)
df['Incident Cost'] = df['Incident Cost'].str.replace(pat =r'[^\w]', repl=r'', regex=True ) #특수문자 제거


bins = [0,10,20,30,40,50]

df['Age_grouping'] = np.digitize(df['Age'], bins)

df

,Date,ID,Gender,Age,Shift,Unnamed: 5,Injury Location,Incident Type,Days Lost,Incident Cost,Age_grouping
0,2020-01-01,438312,Male,25,Night,True,Trunk,Burn,0.0,5000,3
1,2020-01-03,438333,Female,18,Day,True,Abdomen,Cut,0.0,4994,2
2,2020-01-03,438334,Male,35,Day,True,Back,Lifting,5.0,4969,4
3,2020-01-04,438345,Female,50,Day,True,Legs,Lifting,2.5,4947,6
4,2020-01-07,438376,Male,25,Day,True,Abdomen,Lifting,3.0,4940,3
...,...,...,...,...,...,...,...,...,...,...,...
509,2022-06-26,44738511,Female,31,Night,True,Back,Lifting,0.0,0,4
510,2022-06-27,44739512,Male,31,Night,True,Back,Slip/trip,0.0,0,4
511,2022-06-28,44740513,Male,22,Night,True,Abdomen,Crush & Pinch,0.0,0,3
512,2022-06-28,44740514,Male,31,Night,True,Hands,Lifting,0.0,0,4


### 1.1 기본적인 전처리 한 다음에 데이터 분석을 하는 것이 일반적이라 이렇게 정리했음

In [11]:
df.drop(['Date','ID','Unnamed: 5'], axis = 1, inplace = True) # 안쓰는 데이터 1차 제거 

# 라벨링 처리 : 범주형 데이터를 수치형 데이터로 변환
df.loc[df['Gender']== 'Male','Gender'] = 0 #Gender열의 Male 데이터를 0으로 표시
df.loc[df['Gender']== 'Female','Gender'] = 1 #Gender열의 Female 데이터를 1으로 표시
df.loc[df['Shift']== 'Day','Shift'] = 0 #Shift열의 Day 데이터를 0으로 표시
df.loc[df['Shift']== 'Night','Shift'] = 1 #Shift열의 Night 데이터를 1으로 표시
df

,Gender,Age,Shift,Injury Location,Incident Type,Days Lost,Incident Cost,Age_grouping
0,0,25,1,Trunk,Burn,0.0,5000,3
1,1,18,0,Abdomen,Cut,0.0,4994,2
2,0,35,0,Back,Lifting,5.0,4969,4
3,1,50,0,Legs,Lifting,2.5,4947,6
4,0,25,0,Abdomen,Lifting,3.0,4940,3
...,...,...,...,...,...,...,...,...
509,1,31,1,Back,Lifting,0.0,0,4
510,0,31,1,Back,Slip/trip,0.0,0,4
511,0,22,1,Abdomen,Crush & Pinch,0.0,0,3
512,0,31,1,Hands,Lifting,0.0,0,4


### 1.1.1 Injury Location, Incident Type 변환하기

In [12]:
Inj_Loc = df['Injury Location'].unique()
Inc_Type = df['Incident Type'].unique()

In [13]:
# map_Injury = {'Trunk' : 0, 'Abdomen' : 1, 'Back' : 2, 'Legs' : 3, 'Neck':4, 'Head' : 5, 'Multiple':6,
#       'Eye':7, 'Arms': 8, 'Hands' : 9, 'Feet' : 10} : 이것도 데이터 라벨링
# df['target'] = df['Injury Location'].map(map_Injury) : Injury Location을 Target으로 바꿈
# df

In [14]:
Inj_Loc_num_set = {}
Inc_Type_num_set = {}

for i in range(len(Inj_Loc)):
    Inj_Loc_num_set[Inj_Loc[i]] = i
    
for i in range(len(Inc_Type)):
    Inc_Type_num_set[Inc_Type[i]] = i

print(Inj_Loc_num_set)
print(Inc_Type_num_set)

{'Trunk': 0, 'Abdomen': 1, 'Back': 2, 'Legs': 3, 'Neck': 4, 'Head': 5, 'Multiple': 6, 'Eye': 7, 'Arms': 8, 'Hands': 9, 'Feet': 10}
{'Burn': 0, 'Cut': 1, 'Lifting': 2, 'Fall': 3, 'Falling object': 4, 'Crush & Pinch': 5, 'Slip/trip': 6, 'Vehicle': 7, 'Equipment': 8}


In [15]:
df['Injury Location'] = df['Injury Location'].map(Inj_Loc_num_set)
df['Incident Type'] = df['Incident Type'].map(Inc_Type_num_set)

df

,Gender,Age,Shift,Injury Location,Incident Type,Days Lost,Incident Cost,Age_grouping
0,0,25,1,0,0,0.0,5000,3
1,1,18,0,1,1,0.0,4994,2
2,0,35,0,2,2,5.0,4969,4
3,1,50,0,3,2,2.5,4947,6
4,0,25,0,1,2,3.0,4940,3
...,...,...,...,...,...,...,...,...
509,1,31,1,2,2,0.0,0,4
510,0,31,1,2,6,0.0,0,4
511,0,22,1,1,5,0.0,0,3
512,0,31,1,9,2,0.0,0,4


### 1.2 이후부터는 데이터 전처리 과정

In [16]:
scaler = MinMaxScaler()
df[['scaled_DaysLost','scaled_IncidentCost']] = scaler.fit_transform(df[['Days Lost','Incident Cost']])
df

,Gender,Age,Shift,Injury Location,Incident Type,Days Lost,Incident Cost,Age_grouping,scaled_DaysLost,scaled_IncidentCost
0,0,25,1,0,0,0.0,5000,3,0.0,1.0000
1,1,18,0,1,1,0.0,4994,2,0.0,0.9988
2,0,35,0,2,2,5.0,4969,4,1.0,0.9938
3,1,50,0,3,2,2.5,4947,6,0.5,0.9894
4,0,25,0,1,2,3.0,4940,3,0.6,0.9880
...,...,...,...,...,...,...,...,...,...,...
509,1,31,1,2,2,0.0,0,4,0.0,0.0000
510,0,31,1,2,6,0.0,0,4,0.0,0.0000
511,0,22,1,1,5,0.0,0,3,0.0,0.0000
512,0,31,1,9,2,0.0,0,4,0.0,0.0000


### 1.3 실질적으로 데이터를 제거하는 부분 (이부분에 지우고 싶은 데이터 지우세요)

In [17]:
df.drop(['Age','Days Lost','Incident Cost'], axis = 1, inplace = True)

#시계열 데이터 :  시간에 따른 연속성
df.sort_values('Age_grouping', ascending=True)

df

,Gender,Shift,Injury Location,Incident Type,Age_grouping,scaled_DaysLost,scaled_IncidentCost
0,0,1,0,0,3,0.0,1.0000
1,1,0,1,1,2,0.0,0.9988
2,0,0,2,2,4,1.0,0.9938
3,1,0,3,2,6,0.5,0.9894
4,0,0,1,2,3,0.6,0.9880
...,...,...,...,...,...,...,...
509,1,1,2,2,4,0.0,0.0000
510,0,1,2,6,4,0.0,0.0000
511,0,1,1,5,3,0.0,0.0000
512,0,1,9,2,4,0.0,0.0000


In [18]:
df.shape

(472, 7)

#### 일단 에러 뜬 이유는 윗부분에서 drop을 시킨 것들을 형이 다시 당겨온 부분이 있음
#### 형이 만든 코드는 그냥 주석처리해뒀음

In [19]:
# col_names = ['Gender','Age', 'Shift', 'Injury Location','Incident Type', 'Days Lost', 'Incident Cost']

# df.columns = col_names ### 이 부분은 위에처럼 정리를 하면 필요가 없는 부분
## why? = 열이 정리될 때 열의 명도 같이 당겨저서 오기 때문
# # 연속형 변수와 범주형 변수를 구분. 
# df.info()

In [20]:
# 연속형 변수 6개. 
df.columns

df.describe().T  ## 이거 뭐때문에 한거임?


,count,mean,std,min,25%,50%,75%,max
Injury Location,472.0,5.057203,3.174158,0.0,2.0,5.0000,8.00000,10.0
Incident Type,472.0,3.826271,2.591737,0.0,2.0,4.0000,6.00000,8.0
Age_grouping,472.0,4.243644,1.214719,2.0,3.0,4.0000,5.00000,6.0
scaled_DaysLost,472.0,0.144915,0.287783,0.0,0.0,0.0000,0.02500,1.0
scaled_IncidentCost,472.0,0.272070,0.316576,0.0,0.0,0.0999,0.49535,1.0


#### 이부분은 그냥 내가 몰라서 확인해봤음

In [21]:
categorical = [var for var in df.columns if df[var].dtype=='O'] # if df[var] is object type
print(categorical)

['Gender', 'Shift']


##  2. 데이터 전처리 이후 검수 과정

## 2.1 data type이 object (str) 값인 부분을 확인하는 파트

In [22]:
categorical = [var for var in df.columns if df[var].dtype=='O'] # if df[var] is object type
df[categorical].head()

,Gender,Shift
0,0,1
1,1,0
2,0,0
3,1,0
4,0,0


In [23]:
# 각 인자 요소 비율
for var in categorical:
     print(df[var].value_counts()/np.float(len(df)))

0    0.883475
1    0.116525
Name: Gender, dtype: float64
1    0.688559
0    0.311441
Name: Shift, dtype: float64


#### 이 부분은 할 이유가 없다고 생각되는게, 이전에 dropna를 이용하여 null값을 제거함
##### 조심해서 나쁠 건 없음

In [24]:
df[categorical].isnull().sum() # 각각 missing value가 어느정도인지 확인한다. 

Gender    0
Shift     0
dtype: int64

In [25]:
# check for cardinality in categorical variables
# more N of label, more cardinality
for var in categorical:
    print(var, ' contains ', len(df[var].unique()), ' labels')

Gender  contains  2  labels
Shift  contains  2  labels


## 2.2 data type이 숫자 값인 부분을 확인하는 파트

In [26]:
numerical = [var for var in df.columns if df[var].dtype !='O']
print('There are {} numerical variables\n'.format(len(numerical)))
print('The numerical variables are :\n\n', numerical)

There are 5 numerical variables

The numerical variables are :

 ['Injury Location', 'Incident Type', 'Age_grouping', 'scaled_DaysLost', 'scaled_IncidentCost']


In [27]:
df[numerical].head()

,Injury Location,Incident Type,Age_grouping,scaled_DaysLost,scaled_IncidentCost
0,0,0,3,0.0,1.0000
1,1,1,2,0.0,0.9988
2,2,2,4,1.0,0.9938
3,3,2,6,0.5,0.9894
4,1,2,3,0.6,0.9880


#### 이 부분은 할 이유가 없다고 생각되는게, 이전에 dropna를 이용하여 null값을 제거함
##### 마찬가지로 조심해서 나쁠 건 없음

In [29]:
df[numerical].isnull().sum()

Injury Location        0
Incident Type          0
Age_grouping           0
scaled_DaysLost        0
scaled_IncidentCost    0
dtype: int64

In [30]:
#Train, Test data set을 나눈다.

In [31]:
X = df.drop(['Incident Type'], axis=1)
y = df['Incident Type']

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [33]:
# check the shape of X_train and X_test
X_train.shape, X_test.shape

((330, 6), (142, 6))

In [34]:
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']
numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

#### 이 부분은 할 이유가 없다고 생각되는게, 이전에 dropna를 이용하여 null값을 제거함
##### 전자과 학생들이 null값에 민간하긴 한가 봄

In [35]:
# print percentage of missing values in the categorical variables in training set
X_train[categorical].isnull().mean()

Gender    0.0
Shift     0.0
dtype: float64

In [36]:
X_train.shape

(330, 6)

In [37]:
X_test.shape

(142, 6)

In [38]:
cols = X_train.columns

#### 이 부분 에러뜨는 이유는, injery type and place를 str 형태로 유지한 상태로 학습시켰기 때문
##### 어제 이야기 했을 때, str 형을 int형으로 labeling 해야 학습이 된다고 이야기했었음

In [39]:
# import Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
# instantiate the classifier 
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 10 decision-trees : 0.0986
